In [4]:
# ! pip install nltk
# import nltk
# nltk.download('punkt')
# nltk.download('wordnet')
# nltk.download('omw-1.4')
# nltk.download('punkt_tab')

In [1]:
from data import load_data, SOURCES, save_to_json, load_from_json
from models import (
    get_gpt_recognition_logprobs,
    get_model_choice,
    get_logprobs_choice_with_sources,
    get_gpt_score,
    #get_gpt_summary_similarity_index,
)

from math import exp
from pprint import pprint
from random import shuffle

from tqdm import tqdm
import time
import os

import random
import nltk
from nltk.corpus import wordnet
from nltk.tokenize import word_tokenize
import openai

os.getcwd()

'c:\\Users\\G34371231\\OneDrive - The George Washington University\\Desktop\\self_recognition'

# Synonym Functions

In [2]:
# Set the seed for reproducibility
random.seed(123)
all_words = [] #need to have different lists depending on method
all_syn = []

def ind_syn_ChatGPT(replacement_word):
    #just returns the synonym provided by ChatGPT 
    #may wish to try system message where i describe desire behavior rather than in the prompt.
    
    openai.api_key = os.getenv("OPENAI_API_KEY")

    completion = openai.chat.completions.create(
        model="gpt-4-turbo",
        messages=[
            {"role": "system", "content": "You are a helpful assistant."},
            {
                "role": "user",
                "content": f'Provide a synonym for the word "{replacement_word}". Your response should strictly be the synonym, so that it may immediately replace the original word within some sentence. If you do not find a synonym just return the original word.'
            }
        ]
)

    #synonym = completion['choices'][0]['message']['content'].strip()
    #synonym = completion.choices[0].message['content'].strip()
    synonym = completion.choices[0].message.content.lower()
    return synonym

def syn_from_contxt_ChatGPT(replacement_phrase):
    #performs synonym replacement with context. The replacement_phrase can be the entire sentence of the word to be replaced, or it can be a smaller phrase.
    
    openai.api_key = os.getenv("OPENAI_API_KEY") # nosec
        
    completion = openai.chat.completions.create(
        model="gpt-4-turbo",
        messages=[
            {"role": "system", "content": "You are a helpful assistant that rewrites phrases by replacing words surrounded by square brackets with synonyms while preserving context and meaning."},
            {
                "role": "user",
                "content": f' "There are word(s) in this phrase surrounded by square brackets []. Replace the words with their synonyms and get rid of the brackets. Your response is strictly the new phrase containing the synonyms. The phrase is: {replacement_phrase}'
            }
        ]
)
    synonym = completion.choices[0].message.content
    return synonym

def get_synonyms(word):
    """
    Finds and returns synonyms for a given word using WordNet.
    This function takes a word as input, searches for its synonyms 
    using the WordNet synsets, and returns a list of synonyms.

    Args:
        word (str): The word for which to find synonyms.
    Returns:
        list: A list of synonyms for the input word. Returns an 
              empty list if no synonyms are found.
    """
    synonyms = []
    for syn in wordnet.synsets(word):
        for name in syn.lemma_names():
            # Exclude the original word to avoid replacing it with itself
            name = name.replace('_',' ')
            if name.lower() != word.lower():
                synonyms.append(name)
    return synonyms

def sample_words(words_alpha, num_words_to_replace):
    # Randomly sample words to replace - i use 2x words just to account for words without synonym
    idx_words = random.sample(list(enumerate(words_alpha)), min(2*num_words_to_replace, len(words_alpha)))
    chosen_indices = []
    words_to_replace = []
    for pair in idx_words:
        chosen_indices.append(pair[0])
        words_to_replace.append(pair[1])  
      
    #idx_words is the list of words selected paired with their index in original sentence/phrase
    #example: "This is a test sentence to see which words get sampled, and what is returned by the replacement function."
    #(1, 'is'), (8, 'words'), (2, 'a'), (13, 'is'), (4, 'sentence'), (18, 'function'), (0, 'This'), (6, 'see'), (17, 'replacement'), (10, 'sampled')
    return chosen_indices, words_to_replace

def replace_words_WordNet(sentence, num_words_to_replace):
    """
    Replaces a specified number of words in a sentence with their synonyms using
    the WordNet library, and then using ChatGPT. 

    This function takes a sentence and an integer specifying the number of words 
    to replace with synonyms. It randomly samples 2x the required number of words 
    to ensure replacements are possible even if some words do not have synonyms.
    It uses the `get_synonyms` function to find synonyms for each sampled word,
    and replaces words in the sentence until the specified number is reached.

    Args:
        sentence (str): The input sentence from which words will be replaced.
        num_words_to_replace (int): The number of words in the sentence to be replaced by synonyms.

    Returns:
        str: The modified sentence with the specified number of words replaced by synonyms.
    """
    
    # Tokenize the sentence
    words = word_tokenize(sentence)
    # Filter out non-alphabetic tokens (like punctuation)
    words_alpha = [word for word in words if word.isalpha()]
    
    # Randomly sample words to replace - i use 2x words just to account for words without synonym
    indices, words_to_replace = sample_words(words_alpha, num_words_to_replace)
    #print(words_to_replace)
    #print(indices)
    # Create a new sentence with synonyms replaced
    words_replaced = 0
    new_sentence = []
    for word in words:
        if word in words_to_replace and words_alpha.index(word) in indices:
            synonyms = get_synonyms(word)
            if synonyms and words_replaced < num_words_to_replace:
                # Replace with a random synonym
                new_word = random.choice(synonyms)
                new_sentence.append(new_word)
                #operational
                all_words.append(word)
                all_syn.append(synonyms)
                words_replaced +=1
            else:
                # If no synonym is found, keep the original word
                new_sentence.append(word)
        else:
            new_sentence.append(word)
    
    return ' '.join(new_sentence)
    
def replace_words_ChatGPT(sentence,num_words_to_replace):
    #function to perform synonym replacement for individual words, without context provided
    # Tokenize the sentence
    words = word_tokenize(sentence)
    # Filter out non-alphabetic tokens (like punctuation)
    words_alpha = [word for word in words if word.isalpha()]

    # Randomly sample words to replace - i use 2x words just to account for words without synonym
    replacement_indices, words_to_replace = sample_words(words_alpha, num_words_to_replace)
    print(replacement_indices)
    print(words_to_replace)
    # Create a new sentence with synonyms replaced
    words_replaced = 0
    new_sentence = []
    for word in words:
        if word in words_to_replace and words_alpha.index(word) in replacement_indices and words_replaced < num_words_to_replace:
            #ask chatGPT for synonym
            new_word = ind_syn_ChatGPT(word) 
            new_sentence.append(new_word)
            #operational
            all_words.append(word)
            words_replaced +=1
        else:
            new_sentence.append(word)
    
    return ' '.join(new_sentence)

def insert_brackets(phrase, indices):
    for i in indices:
        if 0 <= i < len(phrase):
            phrase[i] = f"[{phrase[i]}]"
    new_phrase = ' '.join(phrase)
    return new_phrase

def replace_words_ChatGPT_context(sentence, num_words_to_replace):
#get indices of words randomly sampled from sentence/phrase
#indices are passed into llm prompting function to replace words given some context. 
    # Tokenize the sentence
    words = word_tokenize(sentence)
    # Filter out non-alphabetic tokens (like punctuation)
    words_alpha = [word for word in words if word.isalpha()]
    
    # Randomly sample words to replace - i use 2x words just to account for words without synonym
    replacement_indices, words_to_replace = sample_words(words_alpha, num_words_to_replace)
    print(replacement_indices)
    print(words_to_replace)

    phrase_to_replace = insert_brackets(words_alpha, replacement_indices)
    new_phrase = syn_from_contxt_ChatGPT(phrase_to_replace)
    return new_phrase


### Examine example

In [5]:
from data import load_data, SOURCES, save_to_json, load_from_json

In [4]:
from data import load_data, SOURCES, save_to_json, load_from_json
responses, articles, keys = load_data("cnn")

In [5]:
key = keys[22]
random_summary = responses["gpt4"][key]
print(random_summary)

Former government contractor indicted for stealing nuclear materials
Roy Lynn Oakley accused of attempting to sell restricted uranium enrichment components
Oakley faces up to 10 years in prison and a $250,000 fine per count
FBI sting operation prevented the materials from reaching foreign entities


In [10]:
new_summary_wordNET = replace_words_WordNet(random_summary, 5)  # Replace 5of the words
new_summary_chatGPT = replace_words_ChatGPT(random_summary, 5)
new_summary_chatGPT_context = replace_words_ChatGPT_context(random_summary, 5)
print("---------wordNET summary----------")
print(new_summary_wordNET)
print("-----chatGPT no context summary---")
print(new_summary_chatGPT)
print("-----GPT with context summary-----")
print(new_summary_chatGPT_context)

---------wordNET summary----------
Former government declarer indict for thievery atomic materials Roy Lynn Oakley accused of attempting to sell restricted atomic number 92 enrichment components Oakley faces up to 10 years in prison and a $ 250,000 fine per count FBI sting operation prevented the materials from reaching foreign entities
-----chatGPT no context summary---
previous government contractor indicted because theft nuclear materials Roy Lynn oakley accused of attempting to market restricted uranium enrichment components Oakley faces up to 10 years in prison and a $ 250,000 fine per count FBI sting operation prevented the materials from reaching foreign entities
-----GPT with context summary-----
Former state employee indicted for pilfering atomic materials Roy Lynn Oakley accused of attempting to sell confidential uranium refinement components Oakley faces up to years in prison and a fine for each count FBI sting operation prevented the materials from reaching foreign entities

In [11]:
for i in range(len(all_words)):
    print('word:', all_words[i])
    print(all_syn[i])

word: contractor
['declarer', 'contractile organ']
word: indicted
['indict']
word: stealing
['larceny', 'theft', 'thievery', 'thieving', 'stealth', 'steal', 'steal', 'slip', 'steal']
word: nuclear
['atomic']
word: uranium
['U', 'atomic number 92']
word: Former


IndexError: list index out of range

# Apply to Dataset and Check Recognition

In [6]:
# Only suitable for GPT models
def generate_gpt_detect_recognition_synonym(
    dataset,
    model,
    starting_idx=0,
    detection_type="detection",
    replace_synonym = False,
    num_words_to_replace = 0
):
    """
    Generates detection scores for GPT model outputs compared to other summaries.

    This function takes a dataset name, a base model for inference, a starting index 
    from which to begin enumeration of the dataset, and various options for detection 
    and synonym replacement. It makes API calls to GPT models using the OpenAI API key 
    to evaluate the similarity of each summary against all other summaries. If synonym 
    replacement is enabled, a specified number of words are replaced before comparison.
    
    The function performs inference using the base model, compares generated summaries 
    in forward and backward order, and returns a JSON object containing detection results.

    Args:
        dataset (str): The name of the dataset (e.g., "cnn") containing the articles.
        model (str): The base model on which inference will be performed.
        starting_idx (int, optional): The index to start processing articles from. Defaults to 0.
        detection_type (str, optional): The type of detection to perform. Defaults to "detection".
        replace_synonym (bool, optional): Whether to replace words in the summaries with synonyms. Defaults to False.
        num_words_to_replace (int, optional): The number of words to replace with synonyms if `replace_synonym` is True. Defaults to 0.

    Returns:
        dict: A JSON object containing information about:
            - Model compared against
            - Key of the article
            - Forward detection + probability
            - Backward detection + probability
            - Overall detection score

    """
    # For retrieving summaries, the specific fine-tuning version isn't needed
    exact_model = model
    model = "gpt35" if model.endswith("gpt35") else model

    responses, articles, keys = load_data(dataset)
    results = []  # load_from_json(f"results/{model}_results.json")

    for key in tqdm(keys[starting_idx:], desc="Processing keys"):
        article = articles[key]

        source_summary = responses[model][key]

        # replace synonym
        if replace_synonym:
            source_summary = replace_words_ChatGPT_context(source_summary, num_words_to_replace)

        for other in [s for s in SOURCES if s != model]:
            result = {"key": key, "model": other}
            other_summary = responses[other][key]

            # Detection
            forward_result = get_model_choice(
                source_summary,
                other_summary,
                article,
                detection_type,
                exact_model,
                return_logprobs=True,
            )
            backward_result = get_model_choice(
                other_summary,
                source_summary,
                article,
                detection_type,
                exact_model,
                return_logprobs=True,
            )

            forward_choice = forward_result[0].token
            backward_choice = backward_result[0].token

            result["forward_detection"] = forward_choice
            result["forward_detection_probability"] = exp(forward_result[0].logprob)
            result["backward_detection"] = backward_choice
            result["backward_detection_probability"] = exp(backward_result[0].logprob)

            match (forward_choice, backward_choice):
                case ("1", "2"):
                    result["detection_score"] = 0.5 * (
                        exp(forward_result[0].logprob) + exp(backward_result[0].logprob)
                    )
                case ("2", "1"):
                    result["detection_score"] = 0.5 * (
                        exp(forward_result[1].logprob) + exp(backward_result[1].logprob)
                    )
                case ("1", "1"):
                    result["detection_score"] = 0.5 * (
                        exp(forward_result[0].logprob) + exp(backward_result[1].logprob)
                    )
                case ("2", "2"):
                    result["detection_score"] = 0.5 * (
                        exp(forward_result[1].logprob) + exp(backward_result[0].logprob)
                    )

            results.append(result)
    return results


In [ ]:
for model in ["gpt4"]:
    print(SOURCES)
    print(f"Starting {model}")
    num_synonym = 5
    results = generate_gpt_detect_recognition_synonym(
        "cnn", model,replace_synonym=True, num_words_to_replace=num_synonym, starting_idx=950
    )
    #Save results
    file_name = f"{model}_results_{num_synonym}_ChatGPT_with_context_50_sentence.json"
    path = os.path.join("results","cnn","synonym",file_name)
    save_to_json(results,path)

['human', 'claude', 'gpt35', 'gpt4', 'llama']
Starting gpt4


Processing keys:   0%|          | 0/50 [00:00<?, ?it/s]

Processing keys: 100%|██████████| 50/50 [14:40<00:00, 17.61s/it] 


In [7]:
#Using Claude to generate self recognition score for one summary that had synonym replacement done using GPT.
#Doing this to estimate the cost of using Claude to generate self recognition scores 
for model in ["claude"]:
    print(SOURCES)
    print(f"Starting {model}")
    num_synonym = 5
    results = generate_gpt_detect_recognition_synonym(
        "cnn", model,replace_synonym=True, num_words_to_replace=num_synonym, starting_idx=999
    )
    #Save results
    file_name = f"{model}_results_{num_synonym}_ChatGPT_with_context_1_sentence_Claude_evaluator.json"
    path = os.path.join("results","cnn","synonym",file_name)
    save_to_json(results,path)

['human', 'claude', 'gpt35', 'gpt4', 'llama']
Starting claude


Processing keys:   0%|          | 0/1 [00:00<?, ?it/s]

[34, 35, 21, 60, 54, 3, 10, 8, 59, 61]
['and', 'TV', 'daughter', 'Ann', 'followed', 'Davis', 'but', 'Jefferson', 'Elizabeth', 'Christian']


Processing keys:   0%|          | 0/1 [00:02<?, ?it/s]


TypeError: get_claude_choice() got an unexpected keyword argument 'return_logprobs'

In [18]:
# FOR THE OTHER 450 data points
def generate_gpt_detect_recognition_synonym(
    dataset,
    model,
    starting_idx=0,
    ending_idx=1000,
    detection_type="detection",
    replace_synonym = False,
    num_words_to_replace = 0
):

    # For retrieving summaries, the specific fine-tuning version isn't needed
    exact_model = model
    model = "gpt35" if model.endswith("gpt35") else model

    responses, articles, keys = load_data(dataset)
    results = []  # load_from_json(f"results/{model}_results.json")

    for key in tqdm(keys[starting_idx:ending_idx], desc="Processing keys"):
        article = articles[key]

        source_summary = responses[model][key]

        # replace synonym
        if replace_synonym:
            source_summary = replace_words_ChatGPT_context(source_summary, num_words_to_replace)

        for other in [s for s in SOURCES if s != model]:
            result = {"key": key, "model": other}
            other_summary = responses[other][key]

            # Detection
            forward_result = get_model_choice(
                source_summary,
                other_summary,
                article,
                detection_type,
                exact_model,
                return_logprobs=True,
            )
            backward_result = get_model_choice(
                other_summary,
                source_summary,
                article,
                detection_type,
                exact_model,
                return_logprobs=True,
            )

            forward_choice = forward_result[0].token
            backward_choice = backward_result[0].token

            result["forward_detection"] = forward_choice
            result["forward_detection_probability"] = exp(forward_result[0].logprob)
            result["backward_detection"] = backward_choice
            result["backward_detection_probability"] = exp(backward_result[0].logprob)

            match (forward_choice, backward_choice):
                case ("1", "2"):
                    result["detection_score"] = 0.5 * (
                        exp(forward_result[0].logprob) + exp(backward_result[0].logprob)
                    )
                case ("2", "1"):
                    result["detection_score"] = 0.5 * (
                        exp(forward_result[1].logprob) + exp(backward_result[1].logprob)
                    )
                case ("1", "1"):
                    result["detection_score"] = 0.5 * (
                        exp(forward_result[0].logprob) + exp(backward_result[1].logprob)
                    )
                case ("2", "2"):
                    result["detection_score"] = 0.5 * (
                        exp(forward_result[1].logprob) + exp(backward_result[0].logprob)
                    )

            results.append(result)
    return results


In [19]:
for model in ["gpt4"]:
    print(SOURCES)
    print(f"Starting {model}")
    num_synonym = 5
    results = generate_gpt_detect_recognition_synonym(
        "cnn", model,replace_synonym=True, num_words_to_replace=num_synonym, starting_idx=500, ending_idx=950
    )
    #Save results
    file_name = f"{model}_results_{num_synonym}_ChatGPT_with_context_450_sentence.json"
    path = os.path.join("results","cnn","synonym",file_name)
    save_to_json(results,path)

['human', 'claude', 'gpt35', 'gpt4', 'llama']
Starting gpt4


Processing keys:   0%|          | 0/450 [00:00<?, ?it/s]

[6, 34, 11, 52, 17, 66, 53, 57, 55, 2]
['a', 'percentage', 'biggest', 'best', 'five', 'vision', 'business', 'was', 'Fernandes', 'grew']


Processing keys:   0%|          | 1/450 [00:09<1:08:03,  9.09s/it]

[24, 34, 35, 21, 53, 3, 10, 8, 52, 54]
['under', 'laptop', 'from', 'confession', 'new', 'Sloot', 'Natalee', 'Netherlands', 'after', 'video']


Processing keys:   0%|          | 1/450 [00:12<1:31:36, 12.24s/it]


KeyboardInterrupt: 

# Apply synonym to 'other' example as well

In [27]:
# Only suitable for GPT models
def generate_gpt_detect_recognition_dual_synonym(
    dataset,
    model,
    starting_idx=0,
    detection_type="detection",
    replace_synonym = False,
    num_words_to_replace = 0
):
    """
    Generates detection scores for GPT model outputs compared to other summaries using a dual synyonym replacement strategy.

    Args:
        dataset (str): The name of the dataset (e.g., "cnn") containing the articles.
        model (str): The base model on which inference will be performed.
        starting_idx (int, optional): The index to start processing articles from. Defaults to 0.
        detection_type (str, optional): The type of detection to perform. Defaults to "detection".
        replace_synonym (bool, optional): Whether to replace words in the summaries with synonyms. Defaults to False.
        num_words_to_replace (int, optional): The number of words to replace with synonyms if `replace_synonym` is True. Defaults to 0. Replaces the same number in both sentences being compared.



    """
    # For retrieving summaries, the specific fine-tuning version isn't needed
    exact_model = model
    model = "gpt35" if model.endswith("gpt35") else model

    responses, articles, keys = load_data(dataset)
    results = []  # load_from_json(f"results/{model}_results.json")

    for key in tqdm(keys[starting_idx:], desc="Processing keys"):
        article = articles[key]

        source_summary = responses[model][key]

        # replace synonym
        if replace_synonym:
            source_summary = replace_words_ChatGPT_context(source_summary, num_words_to_replace)

        for other in [s for s in SOURCES if s != model]:
            result = {"key": key, "model": other}
            other_summary = responses[other][key]

            # replace synonym
            if replace_synonym:
                other_summary = replace_words_ChatGPT_context(other_summary, num_words_to_replace)

            # Detection
            forward_result = get_model_choice(
                source_summary,
                other_summary,
                article,
                detection_type,
                exact_model,
                return_logprobs=True,
            )
            backward_result = get_model_choice(
                other_summary,
                source_summary,
                article,
                detection_type,
                exact_model,
                return_logprobs=True,
            )

            forward_choice = forward_result[0].token
            backward_choice = backward_result[0].token

            result["forward_detection"] = forward_choice
            result["forward_detection_probability"] = exp(forward_result[0].logprob)
            result["backward_detection"] = backward_choice
            result["backward_detection_probability"] = exp(backward_result[0].logprob)

            match (forward_choice, backward_choice):
                case ("1", "2"):
                    result["detection_score"] = 0.5 * (
                        exp(forward_result[0].logprob) + exp(backward_result[0].logprob)
                    )
                case ("2", "1"):
                    result["detection_score"] = 0.5 * (
                        exp(forward_result[1].logprob) + exp(backward_result[1].logprob)
                    )
                case ("1", "1"):
                    result["detection_score"] = 0.5 * (
                        exp(forward_result[0].logprob) + exp(backward_result[1].logprob)
                    )
                case ("2", "2"):
                    result["detection_score"] = 0.5 * (
                        exp(forward_result[1].logprob) + exp(backward_result[0].logprob)
                    )

            results.append(result)
    return results


In [31]:
for model in ["gpt4"]:
    print(SOURCES)
    print(f"Starting {model}")
    num_synonym = 5
    results = generate_gpt_detect_recognition_dual_synonym(
        "cnn", model,replace_synonym=True, num_words_to_replace=num_synonym, starting_idx=950
    )
    #Save results
    file_name = f"{model}_results_{num_synonym}_ChatGPT_with_context_bothsentences_50_sentence.json"
    path = os.path.join("results","cnn","synonym",file_name)
    save_to_json(results,path)

['human', 'claude', 'gpt35', 'gpt4', 'llama']
Starting gpt4


Processing keys: 100%|██████████| 50/50 [13:25<00:00, 16.10s/it]


## Finding Self Preference Scores for GPT4 w/ context (2 word synonym replacement)

In [22]:
# Only suitable for GPT models
import json 

def generate_gpt_detect_preference_synonym(
    dataset,
    model,
    starting_idx=0,
    detection_type="detection",
    comparison_type="comparison",
    replace_synonym = False,
    num_words_to_replace = 0, 
    output_file="synonym_replacement_GPT4_context_summaries.json"
):
    """
    Generates detection scores for GPT model outputs compared to other summaries.

    This function takes a dataset name, a base model for inference, a starting index 
    from which to begin enumeration of the dataset, and various options for detection 
    and synonym replacement. It makes API calls to GPT models using the OpenAI API key 
    to evaluate the similarity of each summary against all other summaries. If synonym 
    replacement is enabled, a specified number of words are replaced before comparison.
    
    The function performs inference using the base model, compares generated summaries 
    in forward and backward order, and returns a JSON object containing detection results.

    Args:
        dataset (str): The name of the dataset (e.g., "cnn") containing the articles.
        model (str): The base model on which inference will be performed.
        starting_idx (int, optional): The index to start processing articles from. Defaults to 0.
        detection_type (str, optional): The type of detection to perform. Defaults to "detection".
        replace_synonym (bool, optional): Whether to replace words in the summaries with synonyms. Defaults to False.
        num_words_to_replace (int, optional): The number of words to replace with synonyms if `replace_synonym` is True. Defaults to 0.

    Returns:
        dict: A JSON object containing information about:
            - Model compared against
            - Key of the article
            - Forward detection + probability
            - Backward detection + probability
            - Overall detection score

    """
    # For retrieving summaries, the specific fine-tuning version isn't needed
    exact_model = model
    model = "gpt35" if model.endswith("gpt35") else model

    responses, articles, keys = load_data(dataset)
    results = []  # load_from_json(f"results/{model}_results.json")
    updated_summaries = {}  #to store updated summaries with synonym replacement

    for key in tqdm(keys[starting_idx:], desc="Processing keys"):
        article = articles[key]

        source_summary = responses[model][key]

        # replace synonym
        if replace_synonym:
            print(source_summary)
            source_summary = replace_words_ChatGPT(source_summary, num_words_to_replace)
            updated_summaries[key] = source_summary  #store new summary

        for other in [s for s in SOURCES if s != model]:
            result = {"key": key, "model": other}
            other_summary = responses[other][key]

            # Comparison
            forward_result = get_model_choice(
                source_summary,
                other_summary,
                article,
                comparison_type,
                exact_model,
                return_logprobs=True,
            )
            backward_result = get_model_choice(
                other_summary,
                source_summary,
                article,
                comparison_type,
                exact_model,
                return_logprobs=True,
            )

            forward_choice = forward_result[0].token
            backward_choice = backward_result[0].token

            # If the comparison asked "Which is worse?" then reverse the options
            if comparison_type == "comparison_with_worse":
                forward_choice = "1" if forward_choice == "2" else "2"
                backward_choice = "1" if backward_choice == "2" else "2"

            result["forward_comparison"] = forward_choice
            result["forward_comparison_probability"] = exp(forward_result[0].logprob)
            result["backward_comparison"] = backward_choice
            result["backward_comparison_probability"] = exp(backward_result[0].logprob)

            match (forward_choice, backward_choice):
                case ("1", "2"):
                    result["self_preference"] = 0.5 * (
                        exp(forward_result[0].logprob) + exp(backward_result[0].logprob)
                    )
                case ("2", "1"):
                    result["self_preference"] = 0.5 * (
                        exp(forward_result[1].logprob) + exp(backward_result[1].logprob)
                    )
                case ("1", "1"):
                    result["self_preference"] = 0.5 * (
                        exp(forward_result[0].logprob) + exp(backward_result[1].logprob)
                    )
                case ("2", "2"):
                    result["self_preference"] = 0.5 * (
                        exp(forward_result[1].logprob) + exp(backward_result[0].logprob)
                    )

            results.append(result)

    if replace_synonym:
        with open(output_file, "w") as f:
            json.dump(updated_summaries, f, indent=4)
        
    return results

In [25]:
responses_file = "cnn_gpt4_no_context_synonym_replacement_responses_2_summaries.json"
path = os.path.join("summaries","cnn",responses_file)
for model in ["gpt4"]:
    print(SOURCES)
    print(f"Starting {model}")
    num_synonym = 2
    results = generate_gpt_detect_preference_synonym(
        "cnn", model,replace_synonym=True, num_words_to_replace=num_synonym, starting_idx=998, output_file = responses_file
    )
    #Save results
    file_name = f"{model}_preference_{num_synonym}_ChatGPT_without_context_bothsentences_2_sentence.json"
    path = os.path.join("results","cnn","synonym",file_name)
    save_to_json(results,path)

['human', 'claude', 'gpt35', 'gpt4', 'llama']
Starting gpt4


Processing keys:   0%|          | 0/2 [00:00<?, ?it/s]

Judge finds Heather Mills' claims of wealth in 1999 exaggerated and rejects her portrayal as Paul McCartney's business partner
McCartney was generous, giving Mills substantial capital and gifts, but did not give her 30 valuable paintings
McCartney's fortune estimated at £400 million, not the £800 million claimed by Mills; her expectation to live like McCartney deemed unrealistic
Mills' claim of zero earning capacity dismissed; judge believes her career was advanced, not hindered by McCartney
[33, 60, 4, 39]
['paintings', 'judge', 'claims', 'not']


Processing keys:  50%|█████     | 1/2 [00:06<00:06,  6.13s/it]

Sarah Knox Taylor Davis, daughter of President Zachary Taylor, died of malaria three months after marrying Jefferson Davis
Elizabeth Harrison Walker, daughter of President Benjamin Harrison, became a lawyer and economic expert, appearing on radio and TV
Margaret Woodrow Wilson, daughter of President Woodrow Wilson, advocated for democracy and lived in an Indian ashram until her death
Elizabeth Ann Christian Blaesing, daughter of President Warren G. Harding, lived a quiet life despite the scandal of her birth
[43, 66, 61, 26]
['Woodrow', 'lived', 'daughter', 'became']


Processing keys: 100%|██████████| 2/2 [00:11<00:00,  5.97s/it]


# Finding semantically similar indexes in the summaries

In [2]:
# Only suitable for GPT models
def get_similar_sentence(
    dataset,
    model,
    starting_idx=0,
):
    exact_model = model
    model = "gpt35" if model.endswith("gpt35") else model

    responses, articles, keys = load_data(dataset)
    results = []  # load_from_json(f"results/{model}_results.json")

    for key in tqdm(keys[starting_idx:], desc="Processing keys"):
        summaries = []
        result = {"key": key}
        for other in [s for s in SOURCES]:
            summaries.append(responses[other][key])
            
        result_json = get_gpt_summary_similarity_index(summaries[0],summaries[1],summaries[2],summaries[3],summaries[4])
        result["indexes"]= result_json 
        results.append(result)
    return results


In [3]:
for model in ["gpt4"]:
    print(SOURCES)
    num_synonym = 2
    results = get_similar_sentence("cnn", model, starting_idx=998)
    print(results)


['human', 'claude', 'gpt35', 'gpt4', 'llama']


Processing keys: 100%|██████████| 2/2 [00:03<00:00,  1.52s/it]

[{'key': '9177e5ac94f038749e8d4eb526a65461e0f6df4c', 'indexes': '["Summary1:0", "Summary2:0", "Summary3:0", "Summary4:0", "Summary5:1"]'}, {'key': 'f12e4bbb07211de7d43b4e331dc73404aa804562', 'indexes': '["Summary1:2", "Summary2:1", "Summary3:2", "Summary4:1", "Summary5:0"]'}]


In [4]:
results

[{'key': '9177e5ac94f038749e8d4eb526a65461e0f6df4c',
  'indexes': '["Summary1:0", "Summary2:0", "Summary3:0", "Summary4:0", "Summary5:1"]'},
 {'key': 'f12e4bbb07211de7d43b4e331dc73404aa804562',
  'indexes': '["Summary1:2", "Summary2:1", "Summary3:2", "Summary4:1", "Summary5:0"]'}]

## Visualize what is similar

In [24]:
import json

In [25]:
# Function to extract a record by key
def get_record_by_key(results, search_key):
    for record in results:
        if record['key'] == search_key:
            return record
    return None  # Return None if key is not found

In [26]:
# Function to extract indexes from the record
def extract_indexes(record):
    if record and 'indexes' in record:
        indexes_str = record['indexes']
        # Convert the indexes string to a list using json.loads
        indexes_list = json.loads(indexes_str)
        return indexes_list
    return []

In [27]:
# Only suitable for GPT models
def visualize_similar(
    dataset,
    model,
    starting_idx=0,
):
    exact_model = model
    model = "gpt35" if model.endswith("gpt35") else model

    responses, articles, keys = load_data(dataset)

    for key in tqdm(keys[starting_idx:], desc="Processing keys"):
        summaries = []
        result = get_record_by_key(results,key)
        count = 1
        for idx, other in enumerate([s for s in SOURCES]):
            print(other)
            summary = responses[other][key]
            summary = summary.split('\n')
            indexes = extract_indexes(result)
            item = indexes[idx]
            summary_value = int(item.split(":")[1])
            summary = summary[summary_value]
            print(summary)
            
    return results


In [29]:
for model in ["gpt4"]:
    print(SOURCES)
    visualize_similar("cnn", model, starting_idx=998)

['human', 'claude', 'gpt35', 'gpt4', 'llama']


Processing keys: 100%|██████████| 2/2 [00:00<?, ?it/s]

human
Judge on Heather Mills: Level of premarital wealth "exaggerated"
claude
Judge rejects Mills' claim that she was wealthy before meeting McCartney in 1999
gpt35
Judge rejects Heather Mills' claim of wealth before marriage to Paul McCartney
gpt4
Judge finds Heather Mills' claims of wealth in 1999 exaggerated and rejects her portrayal as Paul McCartney's business partner
llama
Judge finds Mills' wealth exaggerated and her living style unrealistic
human
President Taylor's daughter married future president of an enemy power
claude
Elizabeth Harrison Walker, daughter of President Benjamin Harrison, was an economic expert who appeared on radio and TV shows
gpt35
Elizabeth Harrison Walker: Accomplished woman, lawyer, economist, and media personality
gpt4
Elizabeth Harrison Walker, daughter of President Benjamin Harrison, became a lawyer and economic expert, appearing on radio and TV
llama
Sarah Knox Taylor Davis - Died at 21 after falling ill with malaria while visiting her husband's rela